In [ ]:
!pip install apyori

  Created wheel for apyori: filename=apyori-1.1.2-cp36-none-any.whl size=5975 sha256=45216ce22b500320cffd83cac2234bbc78e0bd30af9e24b46056b9916b6fecd1
  Stored in directory: /root/.cache/pip/wheels/5d/92/bb/474bbadbc8c0062b9eb168f69982a0443263f8ab1711a8cad0
Successfully built apyori


In [ ]:
import os 
import pandas as pd
import numpy as np
from apyori import apriori
from collections import Counter
from datetime import datetime
from itertools import combinations
import matplotlib.pyplot as plt

In [ ]:
# Aisle Dataset
aisles = pd.read_csv('aisles.csv')
aisles.head()

FileNotFoundError: ignored

In [ ]:
# Checking null value
aisles.isna().sum()

In [ ]:
# Department dataset
department = pd.read_csv('departments.csv')
department.head()

In [ ]:
# Checking null value
department.isna().sum()

In [ ]:
# Product dataset
product = pd.read_csv('products.csv')
product.head()

In [ ]:
# checking null value
product.isna().sum()

In [ ]:
#Order dataset
order = pd.read_csv('orders.csv')
order.head()

In [ ]:
# checking null values (NaN Represents order placed only one time)
order.isna().sum()

In [ ]:
# order product prior dataset
oproduct = pd.read_csv('order_products__prior.csv')
oproduct.head()

In [ ]:
# checking null value
oproduct.isna().sum()

In [ ]:
oproduct_t = pd.read_csv('order_products__train.csv')
oproduct_t.head()

## Number of orders by days of week

In [ ]:
# Order in days of week
ordersdow = order.order_dow.value_counts()
ordersdow

In [ ]:
# Graph for number of orders by days of week
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(ordersdow.index, ordersdow)
ax.set_title('Number of orders by days of week')
ax.set_xlabel('day of week')
ax.set_ylabel('number of orders')
plt.show()

In [ ]:
# Taking product id and product name from product dataset
products_name = {x: y for x, y in zip(product.product_id, product.product_name)}
print(products_name)

In [ ]:
#Merging 2 data frame(Order product and products name) into new
oproduct_name = oproduct.copy()
oproduct_name['product_name'] = oproduct_name.product_id.map(lambda x: products_name[x])
oproduct_name

In [ ]:
# 0 represents product ordered first time and 1 indicated product reordered
reordered = pd.crosstab(oproduct_name.product_name, oproduct_name.reordered)
reordered.sort_values(by = 0, ascending=False)

In [ ]:
# Values in Percentage FTP: first time purchased RP: Re purchased
reordered['total'] = reordered.sum(axis = 1)
reordered['FTP'] = reordered[0] / reordered['total']
reordered['RP'] = reordered[1] / reordered['total']
reordered.head()

In [ ]:
# 100 means only purchased first time not purchased again
reordered.sort_values(by = ['FTP', 'total'], ascending = False)[['FTP', 'total']]

In [ ]:
# Percentage of re purchased items
reordered.sort_values(by = ['RP', 'total'], ascending = False)[['RP', 'total']]

In [ ]:
# Products that ordered most
reordered.total.sort_values(ascending=False)

In [ ]:
# first param total product ordered second param total product
porder = sorted(oproduct.product_id.unique())
print(len(porder), len(product))

In [ ]:
# Products that never been ordered
pnotorder = list(product.product_id[~product.product_id.isin(porder)])
[products_name[product] for product in pnotorder]

# Feature engineering
### Predicting whether a product will be reordered or not

In [ ]:
# Merging order products, product, department dataset
merg = pd.merge(oproduct, order, on='order_id', how='left')
merg.head()

In [ ]:
merg1 = pd.merge(merg, product, on='product_id', how = 'left')
merg1.head()

In [ ]:
mergf = pd.merge(merg1, department, on='department_id',how='left')
mergf.head()

In [ ]:
#Getting the total items using the entire dataset 
mergf['total_items'] = merg1.groupby('user_id').size().astype(np.int16)
mergf['total_items'] = mergf['total_items'].replace(np.nan, 0)

In [ ]:
del mergf['eval_set']
del mergf['add_to_cart_order']
del mergf['days_since_prior_order']
del mergf['user_id']
del mergf['product_name']
del mergf['department']

In [ ]:
mergf.describe()

In [ ]:
# Response variable
y=mergf['reordered']

In [ ]:
del mergf['reordered']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(mergf, y, test_size=0.20, random_state=0)

### naive bayes model

In [ ]:
# Fitting classifier to the Training set
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)
# Predicting the Test set results 
y_pred = classifier.predict(X_test)
# Evaluate the results
from sklearn.metrics import accuracy_score
score1 = accuracy_score(y_test,y_pred)
print("Accuracy is ",round(score1*100,2),"%")

### KNN Model

In [ ]:
# Fitting classifier to the Training set
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
# Predicting the Test set results
y_pred = knn.predict(X_test)
# Evaluate the results
from sklearn.metrics import accuracy_score
score = accuracy_score(y_test,y_pred)
print("Accuracy is ",round(score*100,2),"%")

### Support Vector Machine SVM Model

In [ ]:
# Fitting classifier to the Training set
from sklearn import svm
clf = svm.SVC(kernel='linear') # Linear Kernel
clf.fit(X_train, y_train)
# Predicting the Test set results
y_pred = knn.predict(X_test)
# Evaluate the results
from sklearn.metrics import accuracy_score
score = accuracy_score(y_test,y_pred)
print("Accuracy is ",round(score*100,2),"%")

### Decision Tree Classification Model

In [ ]:
# Fitting classifier to the Training set
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf = clf.fit(X_train,y_train)
# Predicting the Test set results
y_pred = knn.predict(X_test)
# Evaluate the results
from sklearn.metrics import accuracy_score
score = accuracy_score(y_test,y_pred)
print("Accuracy is ",round(score*100,2),"%")

### Random Forest Model

In [ ]:
# Fitting classifier to the Training set
from sklearn.ensemble import RandomForestClassifier
clf=RandomForestClassifier(n_estimators=100)
clf.fit(X_train,y_train)
# Predicting the Test set results
y_pred = knn.predict(X_test)
# Evaluate the results
from sklearn.metrics import accuracy_score
score = accuracy_score(y_test,y_pred)
print("Accuracy is ",round(score*100,2),"%")

# Market Basket Analysis

In [ ]:
# creating new data frame for market basket analysis
aprior = order.copy()
aprior.head()

In [ ]:
# dropping unnecessary columns
aprior.drop(['user_id', 'order_id','eval_set'], axis = 1, inplace= True)
aprior.head()

In [ ]:
# covert 'order_number' it to category 
def order_number_categorical(order_number):
  if order_number in range(5):
    return 'order_number_1-5'
  if order_number in range(5, 10):
    return 'order_number_6-10'
  if order_number in range(10, 20):
    return 'order_number_11-20'
  if order_number in range(20, 40):
    return 'order_number_21-40'
  if order_number in range(40, 50):
    return 'order_number_41-50'
  if order_number in range(50, 70):
    return 'order_number_51-70'
  if order_number >= 70:
    return 'order_number_70+'

In [ ]:
aprior.order_number = aprior.order_number.map(order_number_categorical)
aprior.head()

In [ ]:
# conver 'order_dow' to category
def dowcat(dow):
    if dow in [0, 1]:
        return 'weekend'
    else:
        return 'weekday'

In [ ]:
aprior.order_dow = aprior.order_dow.map(dowcat)
aprior.head()

In [ ]:
# converting 'order_hour_of_day' to category
def hcat(hour):
  if hour in range(7):
    return 'early_hours'
  if hour in range(7,10):
    return 'hour_' + str(hour)
  if hour in range(10, 18):
    return 'peak_hours'
  if hour in range(18, 24):
    return 'hour_' + str(hour)

In [ ]:
aprior.order_hour_of_day = aprior.order_hour_of_day.map(hcat)
aprior.head()

In [ ]:
# coverting 'days_since_prior_order' to category
def intcat(interval):
    if np.isnan(interval):
        return 'first_order'
    elif interval in [7, 14, 21]:
        return 'interval_weekly'
    elif interval == 30:
        return 'interval_30+'
    else:
        return 'interval_others'

In [ ]:
aprior.days_since_prior_order = aprior.days_since_prior_order.map(intcat)
aprior.head()

In [ ]:
# Creating array to apply aprior algorithm
aprior_array = aprior.to_numpy()
aprior_array

In [ ]:
# Taking first 5000 fields to calculate market basket to get rid of memory issues
# Total number of orders: 499 Total number of products: 2809
tdf = oproduct[['order_id', 'product_id']][:5000]
n_order = len(set(tdf.order_id))
n_product = len(set(tdf.product_id))
print(n_order, n_product)

In [ ]:
# Counting product frequency based on product id
pfreq = tdf.product_id.value_counts() / n_order
min_support = 0.009
paprior = pfreq[pfreq >= min_support]
print(paprior)

In [ ]:
# To see which order contains which product
taprior= tdf[tdf.product_id.isin(paprior.index)]
taprior

In [ ]:
# Counting order size
order_sizes = taprior.order_id.value_counts()
order_sizes

In [ ]:
# filtering order size to minimum ordered items 2
min_lenght = 2
orders_apriori = order_sizes[order_sizes >= min_lenght]
print(orders_apriori)

In [ ]:
# taking out order id and product id for those customers who purchased more than 2 items
taprior = taprior[taprior.order_id.isin(orders_apriori.index)]
taprior

In [ ]:
# Making product combinations for all orders to identify pattern
transactions_by_order = taprior.groupby('order_id')['product_id']
for order_id, order_list in transactions_by_order:
  print('Order_id:', order_id, '\nOrder list: ', list(order_list))
  product_combinations = combinations(order_list, 2)
  print('Product:')
  print([i for i in product_combinations])
  print('\n')

In [ ]:
# Making combination of all orders min length will be 2 max length will be 5
def product_combinations(tdf, max_length = 5):
  transactions_by_order = tdf.groupby('order_id')['product_id']
  max_length_reference = max_length
  for order_id, order_list in transactions_by_order:
    max_length = min(max_length_reference, len(order_list))
    order_list = sorted(order_list)
    for l in range(2, max_length + 1):
      product_combinations = combinations(order_list, l)
      for combination in product_combinations:
        yield combination

In [ ]:
# Counting frequency for all combinations
combs = product_combinations(taprior)
counter = Counter(combs).items()
combinations_count = pd.Series([x[1] for x in counter], index = [x[0] for x in counter])
combinations_frequency = combinations_count / n_order
print(combinations_frequency)

In [ ]:
# filtering all combinations min_support = 0.01 and min_length = 2
combinations_apriori = combinations_frequency[combinations_frequency >= min_support]
combinations_apriori = combinations_apriori[combinations_apriori.index.map(len) >= min_lenght]
print(combinations_apriori, len(combinations_apriori))

In [ ]:
# dividing all combinations into different columns
product1 = []
product2 = []
product1and2 = []
for c in combinations_apriori.index:
  c_length = len(c)
  for l in range(1, c_length):
    comb = combinations(c, l)
    for a in comb:
      product1and2.append(c)
      b = list(c)
      for e in a:
        b.remove(e)
      b = tuple(b)
      if len(a) == 1:
        a = a[0]
      product1.append(a)
      if len(b) == 1:
        b = b[0]
      product2.append(b)

In [ ]:
# Putting into new dataframe
apriori_df = pd.DataFrame({'product1': product1,
                           'product2': product2,
                           'product1and2': product1and2})

In [ ]:
apriori_df.head()

In [ ]:
# Counting support for all products
support = {**{x: y for x, y in paprior.items()},
           **{x: y for x, y in combinations_frequency.items()}}
support

In [ ]:
# Count lift and confidence for all combinations
apriori_df[['support_1', 'support_2', 'support_12']] = apriori_df[['product1', 'product2', 'product1and2']].applymap(lambda x: support[x])
apriori_df.drop('product1and2', axis = 1, inplace=True)
apriori_df['confidence'] = apriori_df.support_12 / apriori_df.support_1
apriori_df['lift'] = apriori_df.confidence / apriori_df.support_2
apriori_df

In [ ]:
# filter columns
min_confidence = 0.2
min_lift = 1.0
apriori_df = apriori_df[apriori_df.confidence >= min_confidence]
apriori_df = apriori_df[apriori_df.lift >= min_lift]
apriori_df = apriori_df.sort_values(by = 'lift', ascending=False).reset_index(drop = True) #ordering by the lift
apriori_df.head()

In [ ]:
# Converting product id to product names
def convert_product_id_to_name(product_ids):
  if type(product_ids) == int:
    return products_name[product_ids]
  names = []
  for prod in product_ids:
    name = products_name[prod]
    names.append(name)
  names = tuple(names)
  return names

In [ ]:
# Final combination of products with a greater likelihood of purchasing together
apriori_df[['product1', 'product2']] = apriori_df[['product1', 'product2']].applymap(convert_product_id_to_name)
apriori_df